## This notebook is to test whether adaptability features work.

The test code can be found in `test.py`.

In [1]:
import torch.nn as nn
import torch
import utils

device = torch.device("cuda:2")
torch.cuda.is_available(), device

(True, device(type='cuda', index=2))

In [2]:
from models.AdaptableResNetCifar import AdaptableResNetCifar, createModel

depth = 56
num_classes = 10
resnet = createModel(depth, 'CIFAR10', num_classes=num_classes)
resnet = resnet.to(device)

Create ResNet-56 for CIFAR10


In [3]:
resnet.num_parameters, resnet.num_adaptable_params, resnet.min_model_fraction

(853018, 851904, 0.0013059513398310468)

In [4]:
resnet.print_active_blocks()

Layer 0, block 0 is active with 4672 parameters.
Layer 0, block 1 is active with 4672 parameters.
Layer 0, block 2 is active with 4672 parameters.
Layer 0, block 3 is active with 4672 parameters.
Layer 0, block 4 is active with 4672 parameters.
Layer 0, block 5 is active with 4672 parameters.
Layer 0, block 6 is active with 4672 parameters.
Layer 0, block 7 is active with 4672 parameters.
Layer 0, block 8 is active with 4672 parameters.
Layer 1, block 0 is active with 13952 parameters.
Layer 1, block 1 is active with 18560 parameters.
Layer 1, block 2 is active with 18560 parameters.
Layer 1, block 3 is active with 18560 parameters.
Layer 1, block 4 is active with 18560 parameters.
Layer 1, block 5 is active with 18560 parameters.
Layer 1, block 6 is active with 18560 parameters.
Layer 1, block 7 is active with 18560 parameters.
Layer 1, block 8 is active with 18560 parameters.
Layer 2, block 0 is active with 55552 parameters.
Layer 2, block 1 is active with 73984 parameters.
Layer 2, 

In [5]:
import test
active_blocks = resnet.active_blocks

for num_blocks in range(active_blocks + 1):
    test.test_adaptability(resnet, num_blocks, device)

Test completed with no error for 0 blocks.
Test completed with no error for 1 blocks.
Test completed with no error for 2 blocks.
Test completed with no error for 3 blocks.
Test completed with no error for 4 blocks.
Test completed with no error for 5 blocks.
Test completed with no error for 6 blocks.
Test completed with no error for 7 blocks.
Test completed with no error for 8 blocks.
Test completed with no error for 9 blocks.
Test completed with no error for 10 blocks.
Test completed with no error for 11 blocks.
Test completed with no error for 12 blocks.
Test completed with no error for 13 blocks.
Test completed with no error for 14 blocks.
Test completed with no error for 15 blocks.
Test completed with no error for 16 blocks.
Test completed with no error for 17 blocks.
Test completed with no error for 18 blocks.
Test completed with no error for 19 blocks.
Test completed with no error for 20 blocks.
Test completed with no error for 21 blocks.
Test completed with no error for 22 blocks

In [6]:
model_fractions = utils.get_model_fractions(resnet)
for i in range(len(model_fractions)):
    print(f"Number of Layers: {i}, Model Fraction = {100*model_fractions[i]:.2f}")

Number of Layers: 0, Model Fraction = 0.13
Number of Layers: 1, Model Fraction = 0.68
Number of Layers: 2, Model Fraction = 1.23
Number of Layers: 3, Model Fraction = 1.77
Number of Layers: 4, Model Fraction = 2.32
Number of Layers: 5, Model Fraction = 2.87
Number of Layers: 6, Model Fraction = 3.42
Number of Layers: 7, Model Fraction = 3.96
Number of Layers: 8, Model Fraction = 4.51
Number of Layers: 9, Model Fraction = 5.06
Number of Layers: 10, Model Fraction = 6.70
Number of Layers: 11, Model Fraction = 8.87
Number of Layers: 12, Model Fraction = 11.05
Number of Layers: 13, Model Fraction = 13.22
Number of Layers: 14, Model Fraction = 15.40
Number of Layers: 15, Model Fraction = 17.57
Number of Layers: 16, Model Fraction = 19.75
Number of Layers: 17, Model Fraction = 21.93
Number of Layers: 18, Model Fraction = 24.10
Number of Layers: 19, Model Fraction = 30.61
Number of Layers: 20, Model Fraction = 39.29
Number of Layers: 21, Model Fraction = 47.96
Number of Layers: 22, Model Frac

In [7]:
x = torch.rand(1,3,32,32).to(device)
out, activations = test.forward_and_get_activations(resnet, x)
out.shape, activations[0].shape

(torch.Size([1, 10]), torch.Size([1, 16, 32, 32]))

In [8]:
len(activations)

27

In [12]:
def forward_and_final_activation(net: AdaptableResNetCifar, x: torch.Tensor, eval = True):

    activations = []

    def getActivation():
        # the hook signature
        def hook(model, inp, output):
            #print(inp[0].shape)
            output = output.view(output.size(0), -1)
            activations.append(output)
        return hook 

    hook = net.avgpool.register_forward_hook(getActivation())
    
    if eval:
        net.eval()
        
    out = net(x)

    hook.remove()

    return out, activations[0]

x = torch.rand(2,3,32,32).to(device)
out, activations = forward_and_final_activation(resnet, x)
out.shape, activations[0].shape

(torch.Size([2, 10]), torch.Size([2, 64]))